# Decorator Pattern in Python
ein Vortrag von Kevin Bücher

## Definition

**Eine Dekoratorklasse:**

* Ist ein **Adapter** (siehe das Adapter Pattern)
* Sie implementiert die gleiche Schnittstelle wie das Objekt das sie umhüllt
* Sie delegiert Methodenaufrufe an das Objekt, das sie umhüllt

### Zur Erinnerung das Adapter Pattern:
![adapter.png](https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Objektadapter.svg/511px-Objektadapter.svg.png)

---
<sup>(Quelle: https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Objektadapter.svg/511px-Objektadapter.svg.png)</sup>

### Im Vergleich der Dekorator:
![decorator.png](https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Dekorierer.svg/499px-Dekorierer.svg.png)

---
<sup>(Quelle: https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Dekorierer.svg/499px-Dekorierer.svg.png)</sup>

**Der Zweck der Dekoratorklasse ist es:**
* das Verhalten (das das umhüllte Objekt normalerweise implementieren würde, wenn seine Methoden aufgerufen werden) zu **ergänzen**, zu **entfernen** oder **anzupassen**.

**Mit einer Dekoratorklasse kann man:**

* Methodenaufrufe protokollieren, die normalerweise im Stillen ablaufen würden
* Zusätzliche Einstellungen oder Aufräumarbeiten rund um eine Methode durchführen
* Methodenargumente vorverarbeiten
* Rückgabewerte nachbearbeiten
* Aktionen verbieten, die das umhüllte Objekt normalerweise erlauben würde


## Codebeispiel:

In [ ]:
class Component():
    """
    Die Basisschnittstelle von Component definiert Operationen, die durch Decorators geändert werden können.
    """

    def operation(self) -> str:
        pass

In [ ]:
class ConcreteComponent(Component):
    """
    Konkrete Komponenten bieten Standardimplementierungen der Operationen.
    Es kann mehrere Variationen dieser Klassen geben.
    """

    def operation(self) -> str:
        return "ConcreteComponent"

In [ ]:
class Decorator(Component):
    """
    Die Basisklasse Decorator hat die gleiche Schnittstelle wie die anderen Komponenten und dient in erster Linie dazu,
    die Wrapping-Schnittstelle für alle konkreten Decorators zu definieren.
    Die Standard-Implementierung des Wrapping-Codes kann ein Feld zum Speichern einer umhüllten Komponente und die Initiierung dessen sein.
    """

    _component: Component = None

    def __init__(self, component: Component) -> None:
        self._component = component

    @property
    def component(self) -> str:
        """
        Der Decorator delegiert die gesamte Arbeit an die umhüllte Komponente.
        """
        return self._component

    def operation(self) -> str:
        return self._component.operation()

In [ ]:

class ConcreteDecoratorA(Decorator):
    """
    Konkrete Dekorateure rufen das umhüllte Objekt auf und ändern sein Ergebnis in irgendeiner Weise.
    """

    def operation(self) -> str:
        """
        Decorators können die übergeordnete Implementierung der Operation aufrufen, anstatt das umhüllte Objekt direkt aufzurufen.
        Dieser Ansatz vereinfacht die Erweiterung von Dekoratorklassen.
        """
        return f"ConcreteDecoratorA({self.component.operation()})"


In [ ]:
class ConcreteDecoratorB(Decorator):
    """
    Decorators können ihr Verhalten entweder vor oder nach dem Aufruf eines umhüllten Objekts ausführen.
    """

    def operation(self) -> str:
        return f"ConcreteDecoratorB({self.component.operation()})"

In [ ]:
def client_code(component: Component) -> None:
    """
    Der Client-Code arbeitet mit allen Objekten, die die Komponentenschnittstelle verwenden.
    Auf diese Weise kann er unabhängig von den konkreten Klassen der Komponenten bleiben, mit denen er arbeitet.
    """
    print(f"RESULT: {component.operation()}", end="")

In [ ]:
# Auf diese Weise kann der Client-Code die simplen Components unterstützen...
simple = ConcreteComponent()
print("Client: Hier eine einfache Komponente:")
client_code(simple)
print("\n")

In [ ]:
# ...als auch die dekorierte Variante.
#
# Dekoratoren können nicht nur einfache Komponenten, sondern auch die anderen Dekoratoren umhüllen.
decorator1 = ConcreteDecoratorA(simple)
decorator2 = ConcreteDecoratorB(decorator1)
print("Client: Hier eine dekorierte Komponente:")
client_code(decorator2)

## Decorator Methoden in Python
### (Oder: Decorators nach Python Art)

### Grundlegendes zu Funktionen in Python:

**Funktionen als "First-Class" Objekte:**

In [ ]:
def say_hello(name):
    return f"Hello {name}!"

def func(say_hello):
    """
    Methoden können selbst als Argument übermittelt werden
    """
    return say_hello("Bob")

func(say_hello)

**Inner Funktionen:**

In [ ]:
def parent():
    print("Printing from parent")

    def child():
        print("Printing from child")

    child()

parent()
child() # child kann als inner function nicht außerhalb gerufen werden!

**Returning Functions:**

In [ ]:
def parent(num):
    def first_child():
        return "Hello from first child"

    def second_child():
        return "Hello from second child"

    if num == 1:
        return first_child
    else:
        return second_child

print(parent(1))
print(parent(1)()) #Zurückgegebene Funktion muss wieder als Funktion aufgerufen werden

### Inner Function Decorator:

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)
say_whee()

## Decorator in *Pie* Syntax:

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

#Kein extra Wrapping mehr nötig!
my_decorator(say_whee()) # Entspricht: say_whee = my_decorator(say_whee)

### Mehrfachnutzung von Dekoratoren:

In [ ]:
# Im Modul namens decorators:
def do_twice(func):
    """
    Einfacher Decorator
    """
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

**Nun in Verwendung in anderen Modulen:**

In [ ]:
# Decorator aus anderen Modulen importieren
from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

## Dekoratoren mit Argumenten:

In [ ]:
@do_twice
def greet(name):
    print(f"Hello {name}")

greet("World!") #Traceback wegen fehlender Argumentenannahme im Decorator

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        """
        Annahme von mehreren Argumenten
        :param args: Einfache Argumente (optional)
        :param kwargs: Keyword Argumente (optional)
        """
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

greet("World!")


## Introspection von Python Funktionen

In [ ]:
print

In [ ]:
print.__name__

In [ ]:
help(print)

**Nun mit der eigenen Methode say_whee:**

In [ ]:
say_whee.__name__ #Falsche Identität?

In [ ]:
help(say_whee)

### Functools für Informationen über originale Funktion:

In [ ]:
import functools

def do_twice(func):
    @functools.wraps(func) #<--- wrapped die Informationen der eigentlichen Funktion
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")

say_whee.__name__

## Decorator mit Argumenten:

In [ ]:
def repeat(num_times): # <--- Anzahl von Argumenten muss wieder definiert werden
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

@repeat(num_times=4) # <--- Decorator mit Argument
def greet(name):
    print(f"Hello {name}")

greet("Max")

## Noch ein paar reale Beispiele

In [ ]:
import functools
import time

def timer(func):
    """Aufgabe der Laufzeit der timer Funktion"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # Startzeit
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # Endzeit
        run_time = end_time - start_time    # Gesamte Laufzeit
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

waste_some_time(999)

### Beispiel für Debugging Code:

In [ ]:
import functools

def debug(func):
    """Ausgabe der Signatur und des Rückgabewertes der Funktion"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Hey {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

***Nun mit Debug Informationen:***

In [ ]:
make_greeting("Martin")

In [ ]:
make_greeting("Alex", age=11)

## Klassen als Decorators
**Python besitzt bereits Standard Decorators, wie z.B. hier:**

In [ ]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        return self._radius

    @radius.setter
    def radius(self, value):
        self._radius = value

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        return 3.1415926535

In [ ]:
c = Circle(5)
print("Radius: ",c.radius)
print("Pi: ",c.pi())
c = Circle.unit_circle()
print("Factory Radius: ",c.radius)

### Ganze Klasse als Decorator:

In [ ]:
from dataclasses import dataclass

"""
@dataclass sorgt für das Hinzufügen standardisierter Python Decorator Methoden,
wie z.B. __init__ oder __str__
"""
@dataclass
class PlayingCard:
    rank: str
    suit: str

pc = PlayingCard("6","Diamond")
print(pc)

## Quellen:
* https://de.wikipedia.org/wiki/Decorator
* https://de.wikipedia.org/wiki/Adapter_(Entwurfsmuster)
* https://refactoring.guru/design-patterns/decorator/python/example
* https://python-patterns.guide/gang-of-four/decorator-pattern/
* https://realpython.com/primer-on-python-decorators/